In [9]:
import pandas as pd
from sklearn.preprocessing import MaxAbsScaler, PolynomialFeatures

# read datasets
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

y1_name, y2_name, y3_name = "dir_costs", "traffic_costs_s_r", "lost_trips_costs_s_r"
train_y1, train_y2, train_y3 = train_df[y1_name], train_df[y2_name], train_df[y3_name]
test_y1, test_y2, test_y3 = test_df[y1_name], test_df[y2_name], test_df[y3_name]

# scale features
X_train = train_df.drop(columns=[y1_name, y2_name, y3_name])
scaler = MaxAbsScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

X_test = test_df.drop(columns=[y1_name, y2_name, y3_name])
X_test = scaler.transform(X_test)

poly = PolynomialFeatures(2)
X_train = poly.fit_transform(X_train)
X_test = poly.fit_transform(X_test)

<h2> Pick arbitrary sigma</h2>

In [10]:
from GRNN import GRNN
from metrics import print_metrics

grnn_y1 = GRNN(sigma=0.1)
grnn_y1.fit(X_train, train_y1)

print("------ test metrics ------")
print_metrics(test_y1, grnn_y1.predict(X_test))

------ test metrics ------
Mean Squared Error (MSE):              47066930776505.5078125000
Root Mean Squared Error (RMSE):        6860534.2923496496
Mean Absolute Error (MAE):             4595667.3633589875
R-squared (R²):                        0.7554055734
Mean Absolute Percentage Error (MAPE): 0.1151678102
Max Error (ME):                        30258513.5044571832
Median Absolute Error (MedAE):         3018097.9248453975


<h2> Use differential evolution to pick optimal sigma <h2>

In [11]:
from sklearn.metrics import r2_score
from scipy.optimize import differential_evolution

def objective_function(sigma, train_X, train_y, test_X, test_y):
    grnn = GRNN(sigma=sigma[0])
    grnn.fit(train_X, train_y)
    pred_y = grnn.predict(test_X)
    r2 = r2_score(test_y, pred_y)
    return -r2

<h1> Y1 </h1>

In [13]:
optimization_res = differential_evolution(func=objective_function, 
                                          bounds=[(0, 10)], 
                                          updating='deferred', 
                                          workers=10,
                                          disp=True,
                                          tol=0.0000001,
                                          atol=0.0000001,
                                          args=(X_train, train_y1, X_test, test_y1))
print(optimization_res)
print("Optimal sigma_y1: {:.{}f}".format(optimization_res.x[0], 9))

differential_evolution step 1: f(x)= -0.785922
differential_evolution step 2: f(x)= -0.785922
differential_evolution step 3: f(x)= -0.785922
differential_evolution step 4: f(x)= -0.785922
differential_evolution step 5: f(x)= -0.785922
differential_evolution step 6: f(x)= -0.785922
differential_evolution step 7: f(x)= -0.785922
differential_evolution step 8: f(x)= -0.785922
differential_evolution step 9: f(x)= -0.785986
differential_evolution step 10: f(x)= -0.785986
differential_evolution step 11: f(x)= -0.786032
differential_evolution step 12: f(x)= -0.786032
differential_evolution step 13: f(x)= -0.786034
differential_evolution step 14: f(x)= -0.786034
differential_evolution step 15: f(x)= -0.786034
differential_evolution step 16: f(x)= -0.786034
differential_evolution step 17: f(x)= -0.786034
differential_evolution step 18: f(x)= -0.786034
differential_evolution step 19: f(x)= -0.786034
Polishing solution with 'L-BFGS-B'
 message: Optimization terminated successfully.
 success: True

In [12]:
from GRNN import GRNN
from metrics import print_metrics

grnn_y1 = GRNN(sigma=0.150015599)
grnn_y1.fit(X_train, train_y1)

print("------ test metrics ------")
print_metrics(test_y1, grnn_y1.predict(X_test))

------ test metrics ------
Mean Squared Error (MSE):              41173056622046.2421875000
Root Mean Squared Error (RMSE):        6416623.4595810780
Mean Absolute Error (MAE):             4303056.7980895601
R-squared (R²):                        0.7860344831
Mean Absolute Percentage Error (MAPE): 0.1100113465
Max Error (ME):                        28623382.3479329757
Median Absolute Error (MedAE):         2819187.6176807359


<h1> Y2 </h1>

In [14]:
optimization_res = differential_evolution(func=objective_function, 
                                          bounds=[(0, 10)], 
                                          updating='deferred', 
                                          workers=10,
                                          disp=True,
                                          tol=0.0000001,
                                          atol=0.0000001,
                                          args=(X_train, train_y2, X_test, test_y2))
print(optimization_res)
print("Optimal sigma_y1: {:.{}f}".format(optimization_res.x[0], 9))

differential_evolution step 1: f(x)= -0.607447
differential_evolution step 2: f(x)= -0.607447
differential_evolution step 3: f(x)= -0.665792
differential_evolution step 4: f(x)= -0.665792
differential_evolution step 5: f(x)= -0.695125
differential_evolution step 6: f(x)= -0.695125
differential_evolution step 7: f(x)= -0.69875
differential_evolution step 8: f(x)= -0.698834
differential_evolution step 9: f(x)= -0.698835
differential_evolution step 10: f(x)= -0.698835
differential_evolution step 11: f(x)= -0.698835
differential_evolution step 12: f(x)= -0.698836
differential_evolution step 13: f(x)= -0.698836
differential_evolution step 14: f(x)= -0.698836
differential_evolution step 15: f(x)= -0.698836
differential_evolution step 16: f(x)= -0.698836
differential_evolution step 17: f(x)= -0.698836
Polishing solution with 'L-BFGS-B'
 message: Optimization terminated successfully.
 success: True
     fun: -0.6988358742197669
       x: [ 1.430e-01]
     nit: 17
    nfev: 276
     jac: [ 4.44

In [15]:
from GRNN import GRNN
from metrics import print_metrics

grnn_y2 = GRNN(sigma=0.143009114)
grnn_y2.fit(X_train, train_y2)

print("------ test metrics ------")
print_metrics(test_y2, grnn_y2.predict(X_test))

------ test metrics ------
Mean Squared Error (MSE):              69317590701024.6953125000
Root Mean Squared Error (RMSE):        8325718.6297054682
Mean Absolute Error (MAE):             4477929.7778973533
R-squared (R²):                        0.6988358742
Mean Absolute Percentage Error (MAPE): 1.5812035759
Max Error (ME):                        84574949.1043456942
Median Absolute Error (MedAE):         2575372.8375505344


<h1> Y3 </h1>

In [16]:
optimization_res = differential_evolution(func=objective_function, 
                                          bounds=[(0, 10)], 
                                          updating='deferred', 
                                          workers=10,
                                          disp=True,
                                          tol=0.0000001,
                                          atol=0.0000001,
                                          args=(X_train, train_y3, X_test, test_y3))
print(optimization_res)
print("Optimal sigma_y1: {:.{}f}".format(optimization_res.x[0], 9))

differential_evolution step 1: f(x)= -0.381347
differential_evolution step 2: f(x)= -0.381347
differential_evolution step 3: f(x)= -0.492251
differential_evolution step 4: f(x)= -0.492251
differential_evolution step 5: f(x)= -0.520843
differential_evolution step 6: f(x)= -0.520843
differential_evolution step 7: f(x)= -0.520843
differential_evolution step 8: f(x)= -0.520843
differential_evolution step 9: f(x)= -0.520868
differential_evolution step 10: f(x)= -0.520868
differential_evolution step 11: f(x)= -0.520886
differential_evolution step 12: f(x)= -0.520886
differential_evolution step 13: f(x)= -0.520887
differential_evolution step 14: f(x)= -0.520887
differential_evolution step 15: f(x)= -0.520887
differential_evolution step 16: f(x)= -0.520887
differential_evolution step 17: f(x)= -0.520887
differential_evolution step 18: f(x)= -0.520887
differential_evolution step 19: f(x)= -0.520887
differential_evolution step 20: f(x)= -0.520887
Polishing solution with 'L-BFGS-B'
 message: Opti

In [ ]:
from GRNN import GRNN
from metrics import print_metrics

grnn_y3 = GRNN(sigma=0.150015599)
grnn_y3.fit(X_train, train_y3)

print("------ test metrics ------")
print_metrics(test_y3, grnn_y3.predict(X_test))